In [1]:
# Bring in (new) PNC cohort

import pickle
from pathlib import Path
import numpy as np

newdir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/BSNIP'
newdemo = pickle.load(open(f'{newdir}/demographics.pkl', 'rb'))

newfc = dict()

for sub in newdemo['Age_cal']:
    f = f'{newdir}/fc/{sub}_task-unk_fc.npy'
    if not Path(f).exists():
        continue
    p = np.load(f)
    newfc[f'{sub}'] = p

print(len(newfc))

1244


In [2]:
# Angle estimate

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

pi = 3.14
pi2 = 2*pi

def tonp(t):
    return t.detach().cpu().numpy()

class AngleBasis(nn.Module):
    def __init__(self, mixn):
        super(AngleBasis, self).__init__()
        self.mixn = mixn
        self.thetas = nn.Parameter((pi*torch.rand(self.mixn,264)+pi/2).float().cuda())
        self.jitter = nn.Parameter(torch.ones(self.mixn,264).float().cuda())
        
    def project(self):
        with torch.no_grad():
            self.jitter[self.jitter < 0] = 0
            self.jitter[self.jitter > 1] = 1
        
    def phases(self):
        t0 = self.thetas.unsqueeze(2)
        t1 = self.thetas.unsqueeze(1)
        return t0-t1
    
    def jit(self):
        j0 = self.jitter.unsqueeze(2)
        j1 = self.jitter.unsqueeze(1)
        return j0*j1
    
    def ps(self, jitter=True):
        t = self.phases()
        p = torch.cos(t)
        if jitter:
            j = self.jit()
            p = j*p
        return p
    
    def dump(self):
        return dict(mixn=self.mixn, thetas=tonp(self.thetas), jitter=tonp(self.jitter))
    
    def psum(self):
        return torch.mean(self.ps(), axis=0)
    
    def pvec(self):
        a,b = np.triu_indices(264,1)
        p = self.psum()
        return p[a,b]
    
print('Complete')

Complete


In [ ]:
nepochs = 10000
pperiod = 500

def fromnp(x):
    return torch.from_numpy(x).float().cuda()

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

bases = dict()
i = 0

for subtask in newfc:
    x = fromnp(newfc[subtask])

    min_loss = 10
    sav = None

    for _ in range(2):
        basis = AngleBasis(15)
        optim = torch.optim.Adam(basis.parameters(), lr=1e-2, weight_decay=0)

        for e in range(nepochs):
            optim.zero_grad()
            xhat = basis.pvec()
            loss = rmse(xhat, x)
            loss.backward()
            optim.step()
            basis.project()
            if e == nepochs-1 or e % pperiod == 1:
                print(f'{e} {float(loss)}')

        if loss < min_loss:
            print('Saving')
            min_loss = float(loss)
            sav = basis
    
    bases[subtask] = sav.dump()
    i += 1
    
    print(f'Finished {i} {subtask}')
        
print('Complete')

1 0.3214796781539917
501 0.06397851556539536
1001 0.05707249790430069
1501 0.05445459112524986
2001 0.0519028902053833
2501 0.04800492897629738
3001 0.04506663233041763
3501 0.04181844741106033
4001 0.04020330309867859
4501 0.03928052634000778
5001 0.03855552896857262
5501 0.03798395022749901
6001 0.037620238959789276
6501 0.037418656051158905
7001 0.03698625788092613
7501 0.03676120564341545
8001 0.035752296447753906
8501 0.03484494611620903
9001 0.03372638300061226
9501 0.03305095434188843
9999 0.03280297666788101
Saving
1 0.3193817436695099
501 0.062048591673374176
1001 0.05516259744763374
1501 0.05068109184503555
2001 0.04794442653656006
2501 0.04670661315321922
3001 0.045121803879737854
3501 0.04398895055055618
4001 0.04218669980764389
4501 0.04119938984513283
5001 0.04038809984922409
5501 0.03882203996181488
6001 0.0381905622780323
6501 0.03787299990653992
7001 0.03734612464904785
7501 0.03702101856470108
8001 0.036255963146686554
8501 0.03541986644268036
9001 0.03465862944722175

1001 0.056832753121852875
1501 0.05348730459809303
2001 0.05124728009104729
2501 0.04972004517912865
3001 0.04829008877277374
3501 0.04721523076295853
4001 0.04564899578690529
4501 0.04429863765835762
5001 0.043568190187215805
5501 0.042269039899110794
6001 0.041430890560150146
6501 0.040932536125183105
7001 0.04022833704948425
7501 0.03913860395550728
8001 0.03897890821099281
8501 0.0385499969124794
9001 0.0382094644010067
9501 0.03796800971031189
9999 0.03767262026667595
Finished 8 Baltimore.S0316VHT1
1 0.33768564462661743
501 0.06646111607551575
1001 0.05975741147994995
1501 0.05549890175461769
2001 0.05188509821891785
2501 0.049626875668764114
3001 0.04837046563625336
3501 0.047326453030109406
4001 0.04546908289194107
4501 0.04411979764699936
5001 0.04254183545708656
5501 0.041243188083171844
6001 0.0398501381278038
6501 0.0381094366312027
7001 0.03751214221119881
7501 0.036600615829229355
8001 0.03645723685622215
8501 0.035897936671972275
9001 0.03560902923345566
9501 0.0352265499

501 0.06948355585336685
1001 0.062151141464710236
1501 0.05905348062515259
2001 0.05687142536044121
2501 0.05547023192048073
3001 0.05325670540332794
3501 0.0510767437517643
4001 0.049807991832494736
4501 0.04841487109661102
5001 0.04628739878535271
5501 0.04443734139204025
6001 0.0434175506234169
6501 0.042891863733530045
7001 0.04234900698065758
7501 0.04204767569899559
8001 0.04145526885986328
8501 0.04096580669283867
9001 0.04057852923870087
9501 0.04018314182758331
9999 0.03991536423563957
Saving
1 0.3536560535430908
501 0.06794857233762741
1001 0.06102665513753891
1501 0.05797673016786575
2001 0.05520062521100044
2501 0.05321216955780983
3001 0.05231209099292755
3501 0.05034874007105827
4001 0.048840031027793884
4501 0.04732696712017059
5001 0.046744540333747864
5501 0.046161312609910965
6001 0.045512668788433075
6501 0.0446963906288147
7001 0.04402962327003479
7501 0.043373383581638336
8001 0.0429861880838871
8501 0.04236084595322609
9001 0.04205579310655594
9501 0.0419095940887

1001 0.04987557977437973
1501 0.04678936302661896
2001 0.044133227318525314
2501 0.042632751166820526
3001 0.04088452085852623
3501 0.039275363087654114
4001 0.038309674710035324
4501 0.037331487983465195
5001 0.03663152828812599
5501 0.03574546054005623
6001 0.0348784364759922
6501 0.03425426408648491
7001 0.033310841768980026
7501 0.03245818614959717
8001 0.032005395740270615
8501 0.03145721182227135
9001 0.03091820329427719
9501 0.0304407998919487
9999 0.030254682525992393
Saving
Finished 23 Baltimore.S0926USC1
1 0.30717071890830994
501 0.058611996471881866
1001 0.049103476107120514
1501 0.045276857912540436
2001 0.04337122663855553
2501 0.04207882285118103
3001 0.04121330752968788
3501 0.0401475764811039
4001 0.03956612944602966
4501 0.03910111263394356
5001 0.03881227597594261
5501 0.03727034106850624
6001 0.035197123885154724
6501 0.03489432483911514
7001 0.03484373539686203
7501 0.03467538580298424
8001 0.034522250294685364
8501 0.03450794517993927
9001 0.03411730378866196
9501 

501 0.057077743113040924
1001 0.05217264965176582
1501 0.049798980355262756
2001 0.047855254262685776
2501 0.04600300267338753
3001 0.045384179800748825
3501 0.04434852674603462
4001 0.04346846416592598
4501 0.04265656694769859
5001 0.041107177734375
5501 0.040252067148685455
6001 0.03846678137779236
6501 0.037674564868211746
7001 0.03604357689619064
7501 0.035459596663713455
8001 0.035166338086128235
8501 0.034255266189575195
9001 0.033892497420310974
9501 0.033565204590559006
9999 0.03320379555225372
Saving
1 0.327641099691391
501 0.057353001087903976
1001 0.05200276896357536
1501 0.04896588996052742
2001 0.04727732017636299
2501 0.04646268114447594
3001 0.04570358991622925
3501 0.04414759948849678
4001 0.042433250695466995
4501 0.041728753596544266
5001 0.041053369641304016
5501 0.040173426270484924
6001 0.03949904441833496
6501 0.03875472769141197
7001 0.03808875009417534
7501 0.037203557789325714
8001 0.036294400691986084
8501 0.03523440659046173
9001 0.03471275046467781
9501 0.03

...

8001 0.03754347935318947
8501 0.03732989355921745
9001 0.03680405393242836
9501 0.03644672781229019
9999 0.035463348031044006
Saving
1 0.35109367966651917
501 0.07079436630010605
1001 0.06249673664569855
1501 0.058481182903051376
2001 0.05647372826933861
2501 0.053509216755628586
3001 0.050547316670417786
3501 0.04773159325122833
4001 0.04570911079645157
4501 0.044721707701683044
5001 0.04371807351708412
5501 0.04284894838929176
6001 0.040951456874608994
6501 0.03948742151260376
7001 0.037670597434043884
7501 0.036725111305713654
8001 0.03601212427020073
8501 0.03559654951095581
9001 0.03528589382767677
9501 0.035092487931251526
9999 0.034946322441101074
Saving
Finished 721 Dallas.S7261SGS1
1 0.306171715259552
501 0.05918114259839058
1001 0.05264574661850929
1501 0.049978192895650864
2001 0.04809258133172989
2501 0.04711518809199333
3001 0.045612554997205734
3501 0.04293055832386017
4001 0.04172522947192192
4501 0.040405385196208954
5001 0.039731238037347794
5501 0.03922538831830025
60

8001 0.0293646939098835
8501 0.02888134866952896
9001 0.02856023609638214
9501 0.028188588097691536
9999 0.027906447649002075
Finished 728 Dallas.S7402GIJ1
1 0.2792080044746399
501 0.04405353590846062
1001 0.03777911141514778
1501 0.03504425287246704
2001 0.0329517126083374
2501 0.03153537958860397
3001 0.030736926943063736
3501 0.03028245083987713
4001 0.02928728610277176
4501 0.02854565531015396
5001 0.027952296659350395
5501 0.02724924497306347
6001 0.02711361274123192
6501 0.026842089369893074
7001 0.026117665693163872
7501 0.025994161143898964
8001 0.025920966640114784
8501 0.02563624642789364
9001 0.025631004944443703
9501 0.02563038468360901
9999 0.025646230205893517
Saving
1 0.27889394760131836
501 0.04608969762921333
1001 0.039661046117544174
1501 0.03707965463399887
2001 0.03597822040319443
2501 0.03494161739945412
3001 0.03404003009200096
3501 0.03252077475190163
4001 0.031502965837717056
4501 0.030946750193834305
5001 0.03018156625330448
5501 0.02963598445057869
6001 0.0293

7501 0.034848857671022415
8001 0.034413594752550125
8501 0.03380979597568512
9001 0.03317394107580185
9501 0.03281814977526665
9999 0.03228376433253288
Saving
1 0.3468647599220276
501 0.06980208307504654
1001 0.06204362213611603
1501 0.057207200676202774
2001 0.05242590978741646
2501 0.050262074917554855
3001 0.04776326194405556
3501 0.04548707976937294
4001 0.043832581490278244
4501 0.042136695235967636
5001 0.04065860062837601
5501 0.03997311368584633
6001 0.03877771273255348
6501 0.03772374615073204
7001 0.03703192621469498
7501 0.03626309707760811
8001 0.03562438115477562
8501 0.035455770790576935
9001 0.035116322338581085
9501 0.03477136045694351
9999 0.03448672220110893
Finished 736 Dallas.S7897WGB1
1 0.32999923825263977
501 0.0662381574511528
1001 0.05829472094774246
1501 0.05264807492494583
2001 0.050072021782398224
2501 0.04854070395231247
3001 0.046797048300504684
3501 0.04553914815187454
4001 0.04352893307805061
4501 0.04203515872359276
5001 0.04155842214822769
5501 0.041070

7501 0.027710843831300735
8001 0.027524346485733986
8501 0.027497170493006706
9001 0.027080371975898743
9501 0.026962298899888992
9999 0.02695220522582531
Finished 743 Dallas.S8082VTY1
1 0.3151361048221588
501 0.05951366573572159
1001 0.05381869152188301
1501 0.05078089237213135
2001 0.0490046851336956
2501 0.047083064913749695
3001 0.04559456557035446
3501 0.04404710978269577
4001 0.04163969308137894
4501 0.03962431848049164
5001 0.0388367660343647
5501 0.03782790154218674
6001 0.03742590919137001
6501 0.03678792715072632
7001 0.03604244068264961
7501 0.03513837605714798
8001 0.03483578562736511
8501 0.03475518152117729
9001 0.03442282974720001
9501 0.034244801849126816
9999 0.03404298797249794
Saving
1 0.3083764612674713
501 0.06027679890394211
1001 0.05497458204627037
1501 0.05244247242808342
2001 0.04992084950208664
2501 0.047449011355638504
3001 0.04501692205667496
3501 0.04298613220453262
4001 0.041607677936553955
4501 0.040364883840084076
5001 0.03938593342900276
5501 0.03845484

8001 0.03454387187957764
8501 0.03344428911805153
9001 0.032972656190395355
9501 0.03264860063791275
9999 0.03225042670965195
Saving
1 0.3362714946269989
501 0.06636232882738113
1001 0.05913117155432701
1501 0.054873839020729065
2001 0.05106906220316887
2501 0.049435093998909
3001 0.047972921282052994
3501 0.045074183493852615
4001 0.04371797665953636
4501 0.04303824156522751
5001 0.04141635820269585
5501 0.03966803476214409
6001 0.03824538365006447
6501 0.0362313911318779
7001 0.03478791564702988
7501 0.03421136364340782
8001 0.033542752265930176
8501 0.033225592225790024
9001 0.03288209065794945
9501 0.03243517503142357
9999 0.031645387411117554
Saving
Finished 751 Dallas.S8519PWC1
1 0.39940378069877625
501 0.08232669532299042
1001 0.07398655265569687
1501 0.07067084312438965
2001 0.06788381189107895
2501 0.06458736211061478
3001 0.06010553985834122
3501 0.05671662464737892
4001 0.05258329212665558
4501 0.05124945566058159
5001 0.05025165528059006
5501 0.04922406002879143
6001 0.0474

In [8]:
import pickle

pickle.dump(bases, open('/home/anton/Documents/Tulane/Research/BSNIP/AngleBasisLong15.pkl', 'wb'))

print('Complete')